# Maximum common substructure (MCS)

MolecularGraph.js implements several type of MCS for cheminformatics applications.

- Node induced (MCIS) / Edge induced (MCES)
- Connected / disconnected
- Topological constraint (tdMCS)
- Graph-based local similarity (GLS)


In [1]:
import Pkg
Pkg.activate("..")
using MolecularGraph
using MolecularGraph.MolecularGraphModel


#  Download test data from PubChem

pubchemsdf("6047", "Levodopa")
pubchemsdf("74217", "3-Aminocoumarin")
pubchemsdf("6437877", "Cefditoren Pivoxil")
pubchemsdf("5481173", "Ceftazidime")

┌ Info: Recompiling stale cache file /Users/smatsuoka/.julia/compiled/v1.1/MolecularGraph/6vz1i.ji for MolecularGraph [6c89ec66-9cd8-5372-9f91-fabc50dd27fd]
└ @ Base loading.jl:1184


loading: funcgroup.yaml
loading: ring.yaml
loading: biomolecule.yaml
file: Levodopa.mol already exists
file: 3-Aminocoumarin.mol already exists
file: Cefditoren Pivoxil.mol already exists
file: Ceftazidime.mol already exists


## Basic examples

- At first, try MCS functionalities with very small molecules.
- Drop hydrogens from downloaded molecule data by `makehydrogensimplicit`

In [2]:
ldopa = sdftomol(joinpath(pubchem_dir, "Levodopa.mol"))
ldopa = graphmol(makehydrogensimplicit(ldopa))
display("image/svg+xml",  drawsvg(ldopa, 180, 180))

aminocoumarin = sdftomol(joinpath(pubchem_dir, "3-Aminocoumarin.mol"))
aminocoumarin = graphmol(makehydrogensimplicit(aminocoumarin))
display("image/svg+xml",  drawsvg(aminocoumarin, 150, 150))

### Maximum common induced substructure (MCIS)

- `mcismol` method calculates maximum common induced substructure (MCIS).
- `mcismol` returns a tuple `(mapping, status)`.
- `mapping` (type `Dict{Int,Int}`) is a isomorphisim mapping of node(atom) indices between two molecules.
- `nodesubgraph` can be used to retrieve the matched subgraph and then display the substructure.
- `status`(type `Symbol`) shows the calculation status. `:done` means that the calculation was successfully finished.

In [12]:
(mapping, status) = mcismol(ldopa, aminocoumarin)

subg1 = nodesubgraph(ldopa, Set(keys(mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = nodesubgraph(aminocoumarin, Set(values(mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("Status: $(status)")
println("MCIS size: $(length(mapping))")

Status: done
MCIS size: 9


- Default node matcher function `nodematch` compares two atoms by their atomic symbol (`:C`, `:O`, `:N`, ...) and the number of $\pi$ electrons.
- The number of $\pi$ electrons gives as more intrinsic information of 3D geometry of the molecule than aparent bond order that cannot deal with conjugated system of the molecular orbital.
- Other atomic properties (charges, multiplicity, isotope, ...) of atoms are ignored.
- You can customize `nodematch` function and pass it to `mcismol` as an optional argument `nodematcher`.

### Maximum common edge-induced substructure (MCES)

- `mcesmol` calculates maximum common edge-induced substructure (MCES).
- Most chemists may feel that MCES is more intuitive than MCIS.
- `mcesmol` also returns a tuple `(mapping, status)`.
- Note that the result is an mapping of edge indices. So use `edgesubgraph` to show the matched subgraph.

In [5]:
(mapping, status) = mcesmol(ldopa, aminocoumarin)

subg1 = edgesubgraph(ldopa, Set(keys(mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = edgesubgraph(aminocoumarin, Set(values(mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("Status: $(status)")
println("MCES size: $(length(mapping))")

Status: done
MCIS size: 8


### Connected MCS

- By default, `mcesmol` and `mcismol` calculate disconnected MCS.
- Pass keyword argument `connected=True` to them to enable connected MCS mode.
- Connected MCS is far faster than disconnected MCS but tend to be smaller in size.
- As disconnected MCS does not care the spatial relationship among matched fragments, connected MCS can be more appropriate option for some kind of applications.

In [7]:
(mapping, status) = mcismol(ldopa, aminocoumarin, connected=true)

subg1 = nodesubgraph(ldopa, Set(keys(mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = nodesubgraph(aminocoumarin, Set(values(mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("Status: $(status)")
println("Connected MCIS size: $(length(mapping))")

Status: done
Connected MCIS size: 7


In [8]:
(mapping, status) = mcesmol(ldopa, aminocoumarin, connected=true)

subg1 = edgesubgraph(ldopa, Set(keys(mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = edgesubgraph(aminocoumarin, Set(values(mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("Status: $(status)")
println("Connected MCES size: $(length(mapping))")

Status: done
Connected MCES size: 7


## Further applications

- Then, try it with a little larger molecules.

In [15]:
cefditoren = sdftomol(joinpath(pubchem_dir, "Cefditoren Pivoxil.mol"))
cefditoren = graphmol(makehydrogensimplicit(cefditoren))
mol_svg = drawsvg(cefditoren, 200, 200)
display("image/svg+xml",  mol_svg)

ceftazidime = sdftomol(joinpath(pubchem_dir, "Ceftazidime.mol"))
ceftazidime = graphmol(makehydrogensimplicit(ceftazidime))
mol_svg = drawsvg(ceftazidime, 200, 200)
display("image/svg+xml",  mol_svg)

In [16]:
(mapping, status) = mcesmol(cefditoren, ceftazidime, timeout=5)

subg1 = edgesubgraph(cefditoren, Set(keys(mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = edgesubgraph(ceftazidime, Set(values(mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("Status: $(status)")
println("MCES size: $(length(mapping))")

Status: timedout
MCES size: 21


In [7]:
Profile.clear()
@profile mcesmol(mol1, mol2, topological=true, timeout=10)
Profile.print(mincount=10)


Dict{Int64,Int64} with 28 entries:
  30 => 27
  1  => 1
  32 => 29
  2  => 2
  11 => 8
  21 => 17
  43 => 35
  7  => 5
  10 => 7
  42 => 33
  29 => 26
  35 => 31
  19 => 15
  17 => 13
  8  => 6
  22 => 21
  6  => 4
  24 => 23
  28 => 25
  5  => 3
  20 => 16
  23 => 22
  14 => 9
  31 => 28
  27 => 24
  ⋮  => ⋮

In [8]:

subg1 = edgesubgraph(mol1, Set(keys(mcs.mapping)))
mol_svg = drawsvg(subg1, 300, 300)
display("image/svg+xml",  mol_svg)